<a href="https://colab.research.google.com/github/BenBowring/EnergyData/blob/main/DataInvestigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
transacts = drive.CreateFile({'id': '1MPuNhnaCwBdQa4E941TiWvBLX9z74QxN'})
transacts.GetContentFile('transact_data.csv')

In [19]:
mapped_installs = drive.CreateFile({'id': '1vtTG0o8Sfa4S1zwsDQZPjsaY1KYnIhp_'})
mapped_installs.GetContentFile('JanCompTrimmed.xlsx')

In [15]:
full_installs = drive.CreateFile({'id': '1H9osyBVtiS7oMKQP1Mo0guM6lSoxC262'})
full_installs.GetContentFile('AccCompliance.xlsx')

In [9]:
import pandas as pd

In [16]:
transact_df = pd.read_csv('transact_data.csv')

full_df = pd.read_excel('AccCompliance.xlsx')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15,17,18,19,21,22,23,24,25,26,27,28,43,44,45,47,48,49,50,51,52,53,54,58,60,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell F8855 is marked as a date but the serial value 6504288 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell F8903 is marked as a date but the serial value 6506356 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell F8923 is marked as a date but the serial value 5200391 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/usr/

In [20]:
mapped_df = pd.read_excel('JanCompTrimmed.xlsx')

In [21]:
counts = data.groupby('Company name').agg({'ICIS-ID': 'count'}).sort_values('ICIS-ID', ascending = False)

kept_cols = ['Installation ID', 'Installation name', 'Country code',
               'Permit identifier', 'Activity code', 'Nace code', 'Installation city',
               'Account  holder name', 'Company activity', 'TS status',
               'Installation address 1', 'Installation ZIP',
               'Installation city.1']

dropped_cols = [x for x in data.columns if x not in kept_cols]

name_cols = ['Name 1', 'Name 2', 'Name 3']


,ICIS-ID,Installation ID,Installation name,Country code,Permit identifier,Activity code,Nace code,Installation city,Account holder name,Company name,...,Email 1,Phone 1,Name 2,Email 2,Phone 2,Name 3,Email 3,Phone 3,EUTL Link,Comments
0,AT-1,1.0,Calmit Bad Ischl,AT,IKA119,30.0,23.52,Bad Ischl,Calmit GmbH,Baumit,...,t.pollak@mut.or.at,0043 7746 20580,,,,,,,http://ec.europa.eu/clima/ets/ohaDetails.do?ac...,NaN
1,AT-10,10.0,Vetropack Pöchlarn,AT,IGL173,31.0,23.13,Pöchlarn,Vetropack Austria GmbH,Vetropack Group,...,,,,,,Berger DI Dr. Helmut,helmut.berger@allplan.at,0043 1 5053707 94,http://ec.europa.eu/clima/ets/ohaDetails.do?ac...,NaN
2,AT-100,100.0,Verbund FHKW Werndorf 1 Wildon,AT,EEW013,1.0,35.10,Neudorf ob Wildon,VERBUND Thermal Power GmbH & CO KG,Verbund,...,,,,,,VERBUND Thermal Power GmbH & CO KG VERBUND Th...,thermalpower@verbund.com,0043 50313 38015,http://ec.europa.eu/clima/ets/ohaDetails.do?ac...,NaN
3,AT-101,101.0,Verbund FHKW Werndorf 2 Wildon,AT,EEW014,20.0,35.11,Neudorf ob Wildon,VERBUND Thermal Power GmbH & CO KG,Verbund,...,thermalpower@verbund.com,0043 50313 38015,,,,,,,http://ec.europa.eu/clima/ets/ohaDetails.do?ac...,NaN
4,AT-102,102.0,Inn Crystal Glass Braunau,AT,IGL175,7.0,23.13,Braunau am Inn,Walther GmbH,Inn Crystal Glass Braunau,...,cgruss@walther-gmbh.at,0043 7722 881 0,,,,,,,http://ec.europa.eu/clima/ets/ohaDetails.do?ac...,NaN
